# Preprocesamiento de Datos

En este cuaderno, vamos a realizar un preprocesamiento de datos utilizando `scikit-learn`. El preprocesamiento es una fase crítica en cualquier pipeline de machine learning, ya que asegura que los datos estén en un formato adecuado para entrenar modelos. Al final, también veremos cómo almacenar los objetos de preprocesamiento utilizando `joblib` y por qué es importante dentro de las prácticas de MLOps.

## Importación de Librerías
Primero, importamos todas las librerías necesarias. Estas incluyen herramientas de manipulación de datos (`pandas`, `numpy`), visualización (`seaborn`, `matplotlib`), y preprocesamiento (`scikit-learn`).

In [ ]:
# Importamos librerías
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler,  OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from joblib import dump

## Carga de los Datos
Vamos a cargar un conjunto de datos que contiene información demográfica. Este dataset proviene de `data_adults.csv`, un conjunto de datos comúnmente utilizado para predecir el nivel de ingresos basado en características demográficas.

In [ ]:
# Cargamos los datos
Data = pd.read_csv("../Datos/data_adults.csv")
Data.head()

## Exploración de Datos
Es importante conocer la estructura de los datos antes de preprocesarlos. Vamos a visualizar las primeras filas y también la estructura de tipos de datos en cada columna.

In [ ]:
# Vista rápida de los datos
print(Data.info())

## Eliminación de Columnas Irrelevantes
En muchos casos, algunos datos no aportan valor a nuestro modelo o pueden estar altamente correlacionados con otras variables. Aquí, eliminamos la columna `fnlwgt` (un peso final de la muestra) y `education-num` (número de años de educación, que ya está representado en la columna `education`).

In [ ]:
# Eliminamos columnas irrelevantes
Data_cop = Data.drop("fnlwgt", axis=1)
Data_cop = Data_cop.drop("education-num", axis=1)

## Separación de Variables y Objetivo
Dividimos los datos en dos partes: las características (`X`) y la variable objetivo (`y`), que en este caso es la columna `income`.

In [ ]:
X = Data_cop.drop("income", axis=1)
y = Data_cop['income']

## Definición de Estrategia de Preprocesamiento
Aquí definimos diferentes estrategias de preprocesamiento para diferentes tipos de variables:

- Las variables categóricas necesitan ser codificadas utilizando técnicas como OneHotEncoder o OrdinalEncoder.
- Las variables numéricas pueden ser escaladas utilizando StandardScaler o MinMaxScaler.
- También es importante manejar los valores faltantes utilizando imputadores.

In [ ]:
# Definimos las variables categóricas y numéricas
categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns

# Creamos pipelines de preprocesamiento
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Usamos ColumnTransformer para combinar ambas transformaciones
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

## Creación del Pipeline
Ahora combinamos el preprocesamiento en un pipeline. Esto nos permite encadenar las transformaciones de manera eficiente.

In [ ]:
# Definimos el pipeline completo
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

## Entrenamiento de un Modelo con el Pipeline
Ahora que tenemos el pipeline de preprocesamiento configurado, vamos a entrenar un modelo. En este caso, usaremos un **RandomForestClassifier**, un algoritmo robusto y eficiente para tareas de clasificación.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Dividimos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creamos un nuevo pipeline que incluye el preprocesamiento y el modelo
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))])

# Entrenamos el pipeline completo en los datos de entrenamiento
model_pipeline.fit(X_train, y_train)

# Hacemos predicciones en el conjunto de prueba
y_pred = model_pipeline.predict(X_test)

# Evaluamos el rendimiento del modelo
print(classification_report(y_test, y_pred))

## Guardar el Pipeline Completo (Preprocesador + Modelo)
Finalmente, guardamos el pipeline completo (incluyendo el modelo entrenado) en un archivo `joblib`.

In [ ]:
# Guardamos el pipeline completo (preprocesamiento + modelo entrenado)
dump(model_pipeline, 'model_pipeline_rf.joblib')

## ¿Por qué es esto importante en MLOps?

1. **Reproducibilidad**: Guardar el pipeline completo asegura que puedas reproducir exactamente las mismas transformaciones y predicciones en diferentes entornos.
2. **Automatización**: En un entorno MLOps, los pipelines automatizados son clave para la integración continua y el despliegue continuo (CI/CD). Guardar el modelo entrenado con `joblib` facilita esta automatización.
3. **Escalabilidad**: Al tener tanto el preprocesamiento como el modelo entrenado almacenados, puedes aplicarlos a nuevos datos sin necesidad de recalcular cada paso, lo que es esencial cuando se trabaja con grandes volúmenes de datos en producción.
4. **Consistencia**: Evitamos errores derivados de la aplicación inconsistente de transformaciones de datos, garantizando que los datos en producción se traten de la misma forma que en el entrenamiento.